In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from source.TrainDataGeneration import append_ORD_TM,append_ORD_VOL,append_TM_GP,GetGroupDataFrame,append_STEP5

In [2]:
TickDataname='GDF2022_3_27_16_52.csv'
TickGDF=pd.read_csv('CollectedData/'+TickDataname)
Dataname='norm6feat'

TGDF=TickGDF[(TickGDF['TM_GP']>=0)&(TickGDF['TM_GP']<39)]
TGDF=TGDF.set_index(['ORD_DD','ISU_CD','TM_GP'])

new_index = pd.MultiIndex.from_product(TGDF.index.levels)
new_df = TGDF.reindex(new_index)
new_df = new_df.ffill()
new_df=new_df.reset_index()
Train_df=new_df
Train_df.loc[new_df['TM_GP']==0,'시가']=Train_df.loc[new_df['TM_GP']==1,'시가'].values

In [3]:
Train_df['10단계호가합계잔량']=Train_df['매수10단계호가합계잔량']+Train_df['매도10단계호가합계잔량']

In [4]:
qtylist=['ASK_STEP1_BSTORD_RQTY', 'ASK_STEP2_BSTORD_RQTY', 
       'ASK_STEP3_BSTORD_RQTY',  'ASK_STEP4_BSTORD_RQTY', 
       'ASK_STEP5_BSTORD_RQTY', 'BID_STEP1_BSTORD_RQTY', 
       'BID_STEP2_BSTORD_RQTY', 'BID_STEP3_BSTORD_RQTY', 
       'BID_STEP4_BSTORD_RQTY','BID_STEP5_BSTORD_RQTY',
        '매도5단계호가합계잔량', '매수5단계호가합계잔량', '매도10단계호가합계잔량',
       '매수10단계호가합계잔량', '매도총호가잔량', '매수총호가잔량']

Train_df[qtylist]=Train_df[qtylist].div(Train_df['10단계호가합계잔량'],axis=0).values

Train_df['10단계호가합계잔량']=np.log(new_df['10단계호가합계잔량'])

In [5]:
divdict={}
# divdict['10단계호가합계잔량']=qtylist
divdict['시가']=['고가','저가','직전체결가격']
loglist=[]
loglist+=['고가','저가','직전체결가격','시가']
# loglist+=['고가','저가','직전체결가격','시가']
# loglist+=['10단계호가합계잔량']
minmaxnormlist=[]
minmaxnormlist+=['매도총호가잔량', '매수총호가잔량', '고가', '저가', 
           '직전체결가격']

In [6]:
for divcol in divdict:
    Train_df[divdict[divcol]]=Train_df[divdict[divcol]].div(Train_df[divcol],axis=0).values

In [7]:
Train_df[loglist]=np.log(Train_df[loglist])

In [8]:
Train_df[minmaxnormlist]=(Train_df[minmaxnormlist]-Train_df[minmaxnormlist].min())/(Train_df[minmaxnormlist].max()-Train_df[minmaxnormlist].min())

In [9]:
ORDTickDataname='GDF2022_4_12_19_51.csv'
ORDTickGDF=pd.read_csv('CollectedData/'+ORDTickDataname)

In [10]:
ORDTickGDF.columns

Index(['Unnamed: 0', 'ORD_DD', 'ISU_CD', 'MBR_NO', 'BRN_NO', 'ASKBID_TP_CD',
       'MODCANCL_TP_CD', 'ORD_QTY', 'ORD_ACPT_TM', 'TM_GP'],
      dtype='object')

In [11]:
OTGDF=ORDTickGDF[(ORDTickGDF['TM_GP']>=0)&(ORDTickGDF['TM_GP']<39)]
OTGDF=OTGDF[OTGDF['MODCANCL_TP_CD']!=2]

OTGDF['NET_ORD_QTY']=OTGDF['ORD_QTY']*(2*OTGDF['ASKBID_TP_CD']-3)*(2-OTGDF['MODCANCL_TP_CD'])
# OTGDF=OTGDF.set_index(['ORD_DD','ISU_CD','TM_GP'])

In [12]:
groupcolumns=['ORD_DD', 'ISU_CD', 'MBR_NO', 'BRN_NO','TM_GP']
sumcolumns=['NET_ORD_QTY']
meancolumns,lastcolumns=[],[]

In [13]:
OTGDF=GetGroupDataFrame(OTGDF,groupcolumns,sumcolumns,meancolumns,lastcolumns)

In [14]:
feat_cols=['ASK_STEP1_BSTORD_RQTY', 'ASK_STEP2_BSTORD_RQTY', 
       'ASK_STEP3_BSTORD_RQTY',  'ASK_STEP4_BSTORD_RQTY', 
       'ASK_STEP5_BSTORD_RQTY', 'BID_STEP1_BSTORD_RQTY', 
       'BID_STEP2_BSTORD_RQTY', 'BID_STEP3_BSTORD_RQTY', 
       'BID_STEP4_BSTORD_RQTY','BID_STEP5_BSTORD_RQTY',
        '매도5단계호가합계잔량', '매수5단계호가합계잔량', '매도10단계호가합계잔량',
       '매수10단계호가합계잔량', '매도총호가잔량', '매수총호가잔량', '고가', '저가',
           '시가', '직전체결가격','NET_ORD_QTY']

In [15]:
feat_cols=['매도총호가잔량', '매수총호가잔량', '고가', '저가', 
           '직전체결가격','NET_ORD_QTY']

feat_cols=['매도5단계호가합계잔량', '매수5단계호가합계잔량', '매도10단계호가합계잔량',
       '매수10단계호가합계잔량', '매도총호가잔량', '매수총호가잔량', '고가', '저가',
           '시가', '직전체결가격','NET_ORD_QTY']

In [16]:
mbrnlist1=[(5,194),(2,155),(12,100),(17,29),(42,1),(44,1),(50,92),(2,83),(4,10118),(8,298)]
mbrnlist2=[(5,194),(12,100),(2,155),(17,29),(42,1),(44,1),(2,83),(4,10118),(4,9997),(50,91)]
mbrnlist=mbrnlist1+[a for a in mbrnlist2 if a not in mbrnlist1]

In [17]:
Train_df[feat_cols[:-1]]

,매도총호가잔량,매수총호가잔량,고가,저가,직전체결가격
0,0.024334,0.025399,0.096891,0.830602,0.532876
1,0.030896,0.030938,0.097070,0.809086,0.521453
2,0.034993,0.034354,0.097070,0.806399,0.517398
3,0.035894,0.037392,0.097070,0.800107,0.514551
4,0.036188,0.040350,0.097070,0.796053,0.515832
...,...,...,...,...,...
81895,0.089023,0.020885,0.185336,0.798654,0.563496
81896,0.089458,0.020665,0.191938,0.798654,0.571856
81897,0.089199,0.023931,0.191938,0.798654,0.562410
81898,0.092135,0.023293,0.191938,0.798654,0.553966


In [18]:
for mbr,brn in mbrnlist:
    print(mbr,brn)
    OTGDF_temp=OTGDF[(OTGDF['MBR_NO']==mbr) &(OTGDF['BRN_NO']==brn)]
    OTGDF_temp=OTGDF_temp.set_index(['ORD_DD','ISU_CD','TM_GP'])
    OTGDF_temp=OTGDF_temp.reindex(new_index)
    OTGDF_temp['NET_ORD_QTY']=OTGDF_temp['NET_ORD_QTY'].fillna(0)
    Train_df['NET_ORD_QTY']=OTGDF_temp['NET_ORD_QTY'].values
    Train_df['ylabel']=Train_df['NET_ORD_QTY']
    Train_df['ylabel'][:-1]=(2*(Train_df['NET_ORD_QTY']>0)+1*(Train_df['NET_ORD_QTY']==0))[1:]
    Train_df['ylabel'][38::39]=1
    print(Train_df['NET_ORD_QTY'].sum())    

    SaveDirPath='/Data/Min/TransLOB/TrainData/'
    now=datetime.now()
    DataSubfix=str(mbr)+'_'+str(brn)+str(now.year)+'_'+str(now.month)+'_'+str(now.day)
    DataSubfix=str(mbr)+'_'+str(brn)+Dataname
    TrainDataname='Train_ORD'+'_'+DataSubfix
    TrainLabelDataname='Train_ORD_Label_'+'_'+DataSubfix
    np.save(SaveDirPath+TrainDataname,Train_df[feat_cols].values)
    np.save(SaveDirPath+TrainLabelDataname,Train_df['ylabel'])
    print("path:",SaveDirPath+TrainDataname)
    print("path:",SaveDirPath+TrainLabelDataname)
with open(SaveDirPath+'DataInformationORD.txt', "a+") as f:
    f.write("TrainDataname:"+TrainDataname+'\n')
    f.write("TrainLabelDataname:"+TrainLabelDataname+'\n')
    f.write("TickDataname:"+TickDataname+'\n')
    f.write("ORDTickDataname:"+ORDTickDataname+'\n')

    f.write("divdict:"+'\n')
    for col,llist in divdict.items():
        f.write(str(col)+'\t'+str(llist))
    f.write('\n')
    f.write("loglist:"+'\n')
    for col in loglist:
        f.write(str(col)+'\t')
    f.write('\n')

    f.write("feat_cols:"+'\n')
    for col in feat_cols:
        f.write(str(col)+'\t')
    f.write('\n')


5 194


/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ ==

-33328534.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_5_194norm6feat
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__5_194norm6feat
2 155
1011342.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_2_155norm6feat
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__2_155norm6feat
12 100
-27072832.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_12_100norm6feat
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__12_100norm6feat
17 29


/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ ==

-27191045.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_17_29norm6feat
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__17_29norm6feat
42 1
2665621.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_42_1norm6feat
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__42_1norm6feat
44 1
-17303050.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_44_1norm6feat
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__44_1norm6feat
50 92


/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ ==

-2678352.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_50_92norm6feat
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__50_92norm6feat
2 83
-13119612.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_2_83norm6feat
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__2_83norm6feat
4 10118
-13138160.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_4_10118norm6feat
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__4_10118norm6feat
8 298
-3971686.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_8_298norm6feat
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__8_298norm6feat
4 9997
-13028711.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_4_9997norm6feat
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__4_9997norm6feat
50 91
-6831958.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_50_91norm6feat
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__50_91norm6feat


/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ ==

In [19]:
Train_df[feat_cols].values

array([[ 2.43336349e-02,  2.53992877e-02,  9.68911474e-02,
         8.30601734e-01,  5.32875765e-01, -4.06900000e+03],
       [ 3.08956912e-02,  3.09379337e-02,  9.70698418e-02,
         8.09085663e-01,  5.21453060e-01,  9.61000000e+02],
       [ 3.49930489e-02,  3.43537977e-02,  9.70698418e-02,
         8.06399412e-01,  5.17398048e-01, -3.66200000e+03],
       ...,
       [ 8.91990140e-02,  2.39306260e-02,  1.91937764e-01,
         7.98654395e-01,  5.62409812e-01,  1.22800000e+03],
       [ 9.21352883e-02,  2.32933159e-02,  1.91937764e-01,
         7.98654395e-01,  5.53965564e-01, -1.20000000e+03],
       [ 9.16372514e-02,  2.58076317e-02,  1.91937764e-01,
         7.98654395e-01,  5.54496858e-01,  0.00000000e+00]])

In [20]:
Train_df[feat_cols].values.shape

(81900, 6)

In [21]:
Train_df['ylabel'].values

array([2., 0., 2., ..., 0., 1., 1.])

a

SaveDirPath='/Data/Min/TransLOB/TrainData/'
now=datetime.now()
TrainDataname='Train_'+str(now.year)+'_'+str(now.month)+'_'+str(now.day)+'_'+str(now.hour)+'_'+str(now.minute)
TrainLabelDataname='Train_Label_'+str(now.year)+'_'+str(now.month)+'_'+str(now.day)+'_'+str(now.hour)+'_'+str(now.minute)
np.save(SaveDirPath+TrainDataname,Train_df[feat_cols].values)
np.save(SaveDirPath+TrainLabelDataname,increment)
with open(SaveDirPath+'DataInformation.txt', "a+") as f:
    f.write("TrainDataname:"+TrainDataname+'\n')
    f.write("TrainLabelDataname:"+TrainLabelDataname+'\n')
    f.write("TickDataname:"+TickDataname)
    for col in feat_cols:
        f.write(str(col)+'\t')
    f.write('\n')
print("path:",SaveDirPath+TrainDataname)
print("path:",SaveDirPath+TrainLabelDataname)